# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 26 2022 12:30PM,242756,15,8326019,DXM Pharmaceutical Inc.,Released
1,Jul 26 2022 12:15PM,242755,10,Eme-102035,Emerginnova,Released
2,Jul 26 2022 12:15PM,242755,10,Enova-10362,Emerginnova,Released
3,Jul 26 2022 12:15PM,242755,10,Enova-10366,Emerginnova,Released
4,Jul 26 2022 12:15PM,242755,10,Eme-102082,Emerginnova,Released
5,Jul 26 2022 12:08PM,242753,10,MSP212545,"Methapharm, Inc.",Released
6,Jul 26 2022 12:08PM,242753,10,MSP212535,"Methapharm, Inc.",Released
7,Jul 26 2022 12:08PM,242753,10,MSP212537,"Methapharm, Inc.",Released
8,Jul 26 2022 11:57AM,242752,10,MSP212538,"Methapharm, Inc.",Released
9,Jul 26 2022 11:57AM,242752,10,MSP212539,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,242751,Released,2
30,242752,Released,19
31,242753,Released,3
32,242755,Released,4
33,242756,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242751,NaN,NaN,2.0
242752,NaN,NaN,19.0
242753,NaN,NaN,3.0
242755,NaN,NaN,4.0
242756,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242696,45.0,8.0,0.0
242711,15.0,9.0,1.0
242718,10.0,17.0,12.0
242720,0.0,0.0,50.0
242721,4.0,10.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242696,45,8,0
242711,15,9,1
242718,10,17,12
242720,0,0,50
242721,4,10,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242696,45,8,0
1,242711,15,9,1
2,242718,10,17,12
3,242720,0,0,50
4,242721,4,10,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242696,45,8,
1,242711,15,9,1
2,242718,10,17,12
3,242720,,,50
4,242721,4,10,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 26 2022 12:30PM,242756,15,DXM Pharmaceutical Inc.
1,Jul 26 2022 12:15PM,242755,10,Emerginnova
5,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc."
8,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc."
27,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc."
29,Jul 26 2022 11:44AM,242747,19,"AdvaGen Pharma, Ltd"
48,Jul 26 2022 11:36AM,242746,21,"NBTY Global, Inc."
49,Jul 26 2022 11:27AM,242745,10,"Nextsource Biotechnology, LLC"
50,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC"
51,Jul 26 2022 11:01AM,242741,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 26 2022 12:30PM,242756,15,DXM Pharmaceutical Inc.,,,1
1,Jul 26 2022 12:15PM,242755,10,Emerginnova,,,4
2,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",,,3
3,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",,,19
4,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",,,2
5,Jul 26 2022 11:44AM,242747,19,"AdvaGen Pharma, Ltd",,5,14
6,Jul 26 2022 11:36AM,242746,21,"NBTY Global, Inc.",,,1
7,Jul 26 2022 11:27AM,242745,10,"Nextsource Biotechnology, LLC",,,1
8,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC",,,1
9,Jul 26 2022 11:01AM,242741,10,Bio-PRF,,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 12:30PM,242756,15,DXM Pharmaceutical Inc.,1,,
1,Jul 26 2022 12:15PM,242755,10,Emerginnova,4,,
2,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",3,,
3,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",19,,
4,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",2,,
5,Jul 26 2022 11:44AM,242747,19,"AdvaGen Pharma, Ltd",14,5,
6,Jul 26 2022 11:36AM,242746,21,"NBTY Global, Inc.",1,,
7,Jul 26 2022 11:27AM,242745,10,"Nextsource Biotechnology, LLC",1,,
8,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC",1,,
9,Jul 26 2022 11:01AM,242741,10,Bio-PRF,11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 12:30PM,242756,15,DXM Pharmaceutical Inc.,1,,
1,Jul 26 2022 12:15PM,242755,10,Emerginnova,4,,
2,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",3,,
3,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",19,,
4,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 12:30PM,242756,15,DXM Pharmaceutical Inc.,1.0,NaN,NaN
1,Jul 26 2022 12:15PM,242755,10,Emerginnova,4.0,NaN,NaN
2,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",3.0,NaN,NaN
3,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",19.0,NaN,NaN
4,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 12:30PM,242756,15,DXM Pharmaceutical Inc.,1.0,0.0,0.0
1,Jul 26 2022 12:15PM,242755,10,Emerginnova,4.0,0.0,0.0
2,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",3.0,0.0,0.0
3,Jul 26 2022 11:57AM,242752,10,"Methapharm, Inc.",19.0,0.0,0.0
4,Jul 26 2022 11:57AM,242751,10,"Methapharm, Inc.",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2427406,49.0,20.0,45.0
15,1456408,35.0,38.0,0.0
16,485442,0.0,20.0,8.0
19,242747,14.0,5.0,0.0
20,970872,94.0,26.0,25.0
21,242746,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2427406,49.0,20.0,45.0
1,15,1456408,35.0,38.0,0.0
2,16,485442,0.0,20.0,8.0
3,19,242747,14.0,5.0,0.0
4,20,970872,94.0,26.0,25.0
5,21,242746,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,49.0,20.0,45.0
1,15,35.0,38.0,0.0
2,16,0.0,20.0,8.0
3,19,14.0,5.0,0.0
4,20,94.0,26.0,25.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,49.0
1,15,Released,35.0
2,16,Released,0.0
3,19,Released,14.0
4,20,Released,94.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,45.0,0.0,8.0,0.0,25.0,0.0
Executing,20.0,38.0,20.0,5.0,26.0,0.0
Released,49.0,35.0,0.0,14.0,94.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,45.0,0.0,8.0,0.0,25.0,0.0
1,Executing,20.0,38.0,20.0,5.0,26.0,0.0
2,Released,49.0,35.0,0.0,14.0,94.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,45.0,0.0,8.0,0.0,25.0,0.0
1,Executing,20.0,38.0,20.0,5.0,26.0,0.0
2,Released,49.0,35.0,0.0,14.0,94.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()